# Laporan Proyek UAS: Klasifikasi Time Series Audio (Anjing vs Kucing)

**Nama:** Hajjid Rafi Mumtaz  
**NIM:** 230411100078  
**Dataset Source:** [Kaggle - Audio Cats and Dogs](https://www.kaggle.com/datasets/mmoreaux/audio-cats-and-dogs)

---

## 1. Business Understanding (Memahami Bisnis)

Tahap ini berfokus pada pemahaman tujuan proyek dan kebutuhan analisis dari sudut pandang pemecahan masalah.

### 1.1 Latar Belakang Masalah
Data audio merupakan salah satu bentuk data *time series* (deret waktu) di mana sinyal suara berubah seiring berjalannya waktu (amplitudo vs waktu). Dalam konteks pemrosesan sinyal digital, membedakan jenis suara hewan secara otomatis memiliki tantangan tersendiri karena variasi frekuensi, durasi, dan *noise* latar belakang.

### 1.2 Tujuan Proyek (Project Objectives)
Tujuan utama dari proyek ini adalah membangun model *Machine Learning* (atau *Deep Learning*) yang mampu melakukan klasifikasi biner pada data *time series* audio untuk:
1.  **Mendeteksi Pola Suara:** Mempelajari fitur karakteristik dari gelombang suara anjing (gonggongan) dan kucing (meong).
2.  **Otomasi Klasifikasi:** Mengklasifikasikan file audio baru secara akurat ke dalam label "Dog" atau "Cat".

### 1.3 Tujuan Analisis
* Mengekstrak fitur bermakna dari data mentah audio (seperti MFCC atau Spektrogram).
* Membangun model klasifikasi yang memiliki akurasi, presisi, dan recall yang baik.
* Menyediakan model yang siap di-*deploy* ke aplikasi web (Streamlit).

## 2. Data Understanding (Memahami Data / EDA)

Tahap ini bertujuan untuk mengumpulkan data, mendeskripsikan data, serta melakukan eksplorasi data (Exploratory Data Analysis/EDA) untuk mengenali karakteristik sinyal audio.


### 2.1 Pengumpulan Data
Dataset diambil dari Kaggle dengan judul "Audio Cats and Dogs". Dataset ini berisi file audio dalam format `.wav`.
* **Sumber:** Kaggle (mmoreaux/audio-cats-and-dogs)
* **Struktur Folder:** Terbagi menjadi folder `cats_dogs` yang berisi subfolder label atau file tercampur dengan penamaan khusus.

### 2.2 Deskripsi Data
Pada tahap ini, dilakukan pemeriksaan properti fisik dari file audio:
* **Jumlah Sampel:** Menghitung total file audio untuk kucing dan anjing.
* **Durasi Audio:** Memeriksa rata-rata panjang durasi audio (detik).
* **Sample Rate:** Memeriksa frekuensi pengambilan sampel (misalnya 16kHz atau 44.1kHz).
* **Channel:** Memastikan apakah audio mono atau stereo.

### 2.3 Eksplorasi Data (EDA)
Visualisasi dilakukan untuk memahami perbedaan pola *time series* antara kedua kelas:

1.  **Visualisasi Waveform (Time Domain):**
    * Plot Amplitudo vs Waktu.
    * *Observasi:* Melihat bentuk gelombang mentah, apakah suara anjing memiliki amplitudo yang lebih meledak-ledak dibandingkan kucing.
2.  **Visualisasi Spektrogram (Frequency Domain):**
    * Mengubah sinyal waktu menjadi representasi visual frekuensi.
    * *Observasi:* Melihat intensitas frekuensi pada waktu tertentu. Gonggongan anjing biasanya memiliki frekuensi rendah yang kuat, sedangkan kucing mungkin memiliki harmonik di frekuensi lebih tinggi.
3.  **Analisis Keseimbangan Kelas (Class Balance):**
    * Memastikan jumlah data *Cat* dan *Dog* seimbang agar model tidak bias.

## 3. Preprocessing Data

Tahap ini adalah proses mengubah data audio mentah menjadi format yang dapat dipahami oleh algoritma *machine learning*. Karena ini adalah data *time series*, fitur temporal sangat penting.

### 3.1 Data Cleaning & Loading
* Menggunakan *library* `librosa` untuk memuat file audio.
* Menghapus file yang korup atau memiliki durasi terlalu pendek (misal < 1 detik) yang tidak mengandung informasi cukup.

### 3.2 Resampling
* Menyamakan *sample rate* seluruh file audio (misalnya diseragamkan menjadi 22.050 Hz) untuk menjaga konsistensi dimensi data.

### 3.3 Fixing Duration (Padding & Truncating)
Model klasifikasi (terutama Neural Network seperti CNN atau LSTM) membutuhkan input dengan dimensi yang tetap (fixed shape).
* **Truncating:** Jika audio lebih panjang dari durasi target (misal 5 detik), audio akan dipotong.
* **Padding:** Jika audio lebih pendek dari target, akan ditambahkan nilai nol (*zero padding*) hingga mencapai panjang yang ditentukan.

### 3.4 Feature Extraction (Ekstraksi Fitur)
Mengubah data *time series* mentah menjadi fitur numerik yang lebih padat informasi. Metode yang digunakan:
* **Mel-Frequency Cepstral Coefficients (MFCCs):** Teknik ini sangat populer dalam pengolahan suara karena merepresentasikan amplop spektral pendek dari gelombang suara yang menyerupai cara telinga manusia mendengar.
    * *Output:* Matriks 2D (koefisien MFCC x waktu).

### 3.5 Label Encoding
* Mengubah label kategori menjadi numerik:
    * `Cat` -> 0
    * `Dog` -> 1

### 3.6 Train-Test Split
* Membagi dataset menjadi:
    * **Training Set (80%):** Untuk melatih model.
    * **Test Set (20%):** Untuk evaluasi performa model.

## 4. Modeling (Pemodelan Data)

Pada tahap ini, arsitektur *Neural Network* dibangun dan dilatih menggunakan data yang telah diproses.

### 4.1 Skenario Pelatihan
Berdasarkan hasil eksekusi kode (log training), berikut adalah konfigurasi model yang diterapkan:
* **Jumlah Data:** Total 267 sampel audio (213 Training, 54 Testing).
* **Input Shape:** 40 Fitur (Berdasarkan ekstraksi MFCC).
* **Jumlah Epoch:** 50 Iterasi.
* **Batch Size:** 32.

### 4.2 Arsitektur Artificial Neural Network (ANN)
Model disusun menggunakan *Keras Sequential API* dengan struktur layer sebagai berikut untuk menangkap pola non-linear dari fitur MFCC:

1.  **Input Layer:** Menerima vektor 40 fitur MFCC.
2.  **Dense Layer 1:** 256 Neuron, Aktivasi ReLU (Menangkap fitur global).
    * *Dropout (0.3):* Untuk mencegah overfitting.
3.  **Dense Layer 2:** 128 Neuron, Aktivasi ReLU (Memperhalus fitur).
    * *Dropout (0.3):* Untuk mencegah overfitting.
4.  **Dense Layer 3:** 64 Neuron, Aktivasi ReLU.
5.  **Output Layer:** 1 Neuron, Aktivasi **Sigmoid**.
    * *Alasan:* Output berupa probabilitas biner (0 s.d 1) dimana nilai mendekati 0 adalah Kucing, dan mendekati 1 adalah Anjing.

## 5. Evaluation (Evaluasi)

Tahap ini mengevaluasi kinerja model berdasarkan hasil pelatihan selama 50 epoch dan pengujian terhadap data tes (*Unseen Data*).

### 5.1 Analisis Proses Training (Learning Curve)
Berdasarkan log pelatihan (*training history*), diamati perilaku model sebagai berikut:
* **Penurunan Loss:** Model memulai dengan *Loss* yang sangat tinggi (~9.52 pada Epoch 1) dan berhasil turun secara signifikan menjadi stabil di angka ~0.2 - 0.3 pada Epoch ke-50. Ini menunjukkan model berhasil "belajar" dan meminimalkan kesalahan.
* **Peningkatan Akurasi:** Akurasi validasi meningkat dari awal yang fluktuatif (sempat drop ke 38% di Epoch 2) namun kemudian stabil di angka tinggi (>85%) mulai Epoch ke-5 hingga akhir.

### 5.2 Hasil Akhir (Final Metrics)
Evaluasi akhir dilakukan menggunakan 20% data testing (54 file) yang tidak digunakan selama pelatihan.
* **Akurasi Final (Test Accuracy): 88.89%**
* **Kesimpulan Evaluasi:** Model memiliki performa yang sangat baik ("Good Fit") untuk tugas klasifikasi ini, mengingat dataset yang digunakan relatif kecil (267 file). Tingkat akurasi mendekati 90% menunjukkan model mampu membedakan karakteristik suara anjing dan kucing dengan konsisten.

## 6. Deployment (Implementasi)

Untuk memenuhi kebutuhan bisnis (aksesibilitas pengguna), model dideploy menggunakan framework **Streamlit**.

### 6.1 Fitur Aplikasi
Aplikasi yang dibangun (`app.py`) memiliki fitur-fitur berikut untuk mendukung *Decision Support*:

1.  **Sistem Upload:** Pengguna dapat mengunggah file audio `.wav` sembarang.
2.  **Data Understanding Visualization:** Aplikasi menampilkan dua grafik untuk membantu pengguna memahami data audio:
    * **Waveform (Time Domain):** Menampilkan amplitudo suara terhadap waktu.
    * **Mel-Spectrogram (Frequency Domain):** Menampilkan intensitas frekuensi ("trafo") yang menjadi dasar input bagi model untuk membedakan pola suara.
3.  **Real-time Prediction:** Model memberikan label "Kucing" atau "Anjing" beserta **Confidence Score** (Tingkat Keyakinan) dalam persentase.

### 6.2 Bukti Implementasi
Berikut adalah alur penggunaan aplikasi:
1.  User upload file audio.
2.  Sistem melakukan preprocessing (Load 3 detik, Extract MFCC).
3.  Model `.h5` memproses input.
4.  Hasil klasifikasi ditampilkan di layar dashboard.

![Deployment](deploy1.png)
![Deployment](deploy2.png)
![Deployment](deploy3.png)